# Tutorial pyOSRD #6
Creating custom agent

In [22]:
from pyosrd.agents.scheduler_agent import SchedulerAgent
import pandas as pd
from dataclasses import dataclass

## Creating a dummy agent
Create an agent that add 60 seconds of delay at the second zone the first train go throught.

In [23]:

class DummySchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 60s more in second zone"""
        extra_delays = self.ref_schedule.durations * 0.
        extra_delays.iloc[0, 1] = 60.
        return extra_delays


## Test our agent on all scenarii
We can launch our custom agent on all scenarii implemented in the module OSRD.scenarii.

In [24]:
from pyosrd.agents.scheduler_agent import regulate_scenarii_with_agents

agent = DummySchedulerAgent("dummy")
regulate_scenarii_with_agents("all", agent)

[16:07:42,514] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[16:07:42,545] [INFO]              [FullInfra] parsing infra
[16:07:42,613] [INFO]              [FullInfra] adaptation to kotlin
[16:07:42,634] [INFO]              [FullInfra] loading signals
[16:07:42,640] [INFO]              [FullInfra] building blocks
[16:07:42,644] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[16:07:42,657] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.110
[16:07:42,710] [DEBUG]             [Allowance] computing range n°1
[16:07:42,713] [DEBUG]             [Allowance]   computing section n°1
[16:07:42,713] [DEBUG]             [Allowance]   target time = 310.84094999999996
[16:07:42,713] [DEBUG]             [Allowance]     starting attempt 1
[16:07:42,715] [DEBUG]             [Allowance]     envelope time 518.958
[16:07:42,715] [DEBUG]             [Allowance]     starting attempt 2
[16:07:42,715] [DEBUG]             [All

,dummy
c1_delay,360.0
c1y2_2trains_conflict,120.0
c1y2_2trains_no_conflict,120.0
c2y11s_conflict_20_trains,655.0
c2y1y2y_2_trains_no_conflict,30.0
c2y1y2y_2_trains_reorder,449.0
c2y1y2y_2_trains_same_order,224.0


## Compare two agents

The function regulate_scenarii_with_agents can be launched on multiple agents at the same time. It returns a pandas DataFrame containing the ponderated delays
for all scenarii.

An agent can be compared to another one using those scenarii.

In [25]:

class DumberSchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 120s more in second zone"""
        extra_delays = self.ref_schedule.durations * 0.
        extra_delays.iloc[0, 1] = 120.
        return extra_delays

To speed up the notebook execution the next call is done only on one scenario "c1_delay".

In [27]:

agent_dumber = DumberSchedulerAgent("dumber")
regulate_scenarii_with_agents("c1_delay", [agent, agent_dumber])

[16:08:44,133] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[16:08:44,171] [INFO]              [FullInfra] parsing infra
[16:08:44,232] [INFO]              [FullInfra] adaptation to kotlin
[16:08:44,254] [INFO]              [FullInfra] loading signals
[16:08:44,259] [INFO]              [FullInfra] building blocks
[16:08:44,263] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[16:08:44,279] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.152
[16:08:44,328] [DEBUG]             [Allowance] computing range n°1
[16:08:44,330] [DEBUG]             [Allowance]   computing section n°1
[16:08:44,330] [DEBUG]             [Allowance]   target time = 310.84094999999996
[16:08:44,330] [DEBUG]             [Allowance]     starting attempt 1
[16:08:44,332] [DEBUG]             [Allowance]     envelope time 518.958
[16:08:44,332] [DEBUG]             [Allowance]     starting attempt 2
[16:08:44,332] [DEBUG]             [All

,dummy,dumber
c1_delay,360.0,480.0
